In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
%matplotlib inline
%load_ext Cython

pd.set_option("display.max_columns",100)

In [48]:
!ls ../../datasets/??wiki_*.bz2

../../datasets/dewiki_20170420_reverted_bot2bot.tsv.bz2
../../datasets/enwiki_20161201_reverted_bot2bot.tsv.bz2
../../datasets/eswiki_20170420_reverted_bot2bot.tsv.bz2
../../datasets/frwiki_20170420_reverted_bot2bot.tsv.bz2
../../datasets/jawiki_20170420_reverted_bot2bot.tsv.bz2
../../datasets/ptwiki_20170420_reverted_bot2bot.tsv.bz2
../../datasets/zhwiki_20170420_reverted_bot2bot.tsv.bz2


In [5]:
!bunzip2 -k ../../datasets/??wiki_20170420_reverted_bot2bot.tsv.bz2
!bunzip2 -k ../../datasets/enwiki_20161201_reverted_bot2bot.tsv.bz2

bunzip2: Output file ../../datasets/enwiki_20161201_reverted_bot2bot.tsv already exists.


## Data processing

In [49]:
filelist = glob.glob("../../datasets/??wiki_20170420*.tsv")
filelist.append(glob.glob("../../datasets/enwiki_20161201_reverted_bot2bot.tsv")[0])
filelist

['../../datasets/eswiki_20170420_reverted_bot2bot.tsv',
 '../../datasets/frwiki_20170420_reverted_bot2bot.tsv',
 '../../datasets/dewiki_20170420_reverted_bot2bot.tsv',
 '../../datasets/jawiki_20170420_reverted_bot2bot.tsv',
 '../../datasets/ptwiki_20170420_reverted_bot2bot.tsv',
 '../../datasets/zhwiki_20170420_reverted_bot2bot.tsv',
 '../../datasets/enwiki_20161201_reverted_bot2bot.tsv']

In [50]:
df_dict = {}
for filename in filelist:
    lang_code = filename[15:17]
    df_dict[lang_code] = pd.read_csv(filename, sep="\t")
    df_dict[lang_code] = df_dict[lang_code].drop_duplicates()

In [51]:
for lang, lang_df in df_dict.items():
    print(lang, len(lang_df))

de 133711
en 500553
zh 51536
es 88949
pt 70973
fr 96508
ja 74577


In [52]:
df_dict['en'][0:2].transpose()

,0,1
rev_id,273691771,136526894
rev_timestamp,20090227173507,20070607044209
rev_user,6505923,4534303
rev_user_text,Kbdankbot,PbBot
rev_page,5040439,3046554
rev_sha1,qj45ne2z4yfexmpaz5wfnbm2yrmqt4j,3xtnw7u4w9h6cg1smw97mqnr1en6a55
rev_minor_edit,False,False
rev_deleted,False,False
rev_parent_id,2.59117e+08,1.20932e+08
archived,False,False


### Combining into one dataframe

In [53]:
df_all = df_dict['en'].copy()
df_all = df_all.drop(df_all.index, axis=0)

for lang, lang_df in df_dict.items():
    lang_df['language'] = lang
    df_all = pd.concat([df_all, lang_df])

In [54]:
df_all['language'].value_counts()

en    500553
de    133711
fr     96508
es     88949
ja     74577
pt     70973
zh     51536
Name: language, dtype: int64

### Namespace type

In [55]:
def namespace_type(item):
    if int(item) == 0:
        return 'article'
    elif int(item) == 14:
        return 'category'
    elif int(item) % 2 == 1:
        return 'other talk'
    else:
        return 'other page'

In [56]:
df_all['namespace_type'] = df_all['page_namespace'].apply(namespace_type)

In [57]:
df_all['namespace_type'].value_counts()

article       525818
other talk    201439
category      175699
other page    113851
Name: namespace_type, dtype: int64

### Datetime parsing

In [58]:
def get_year(timestamp):
    return timestamp.year

In [59]:
df_all['reverting_timestamp_dt'] = pd.to_datetime(df_all['reverting_timestamp'], format="%Y%m%d%H%M%S")

df_all['reverted_timestamp_dt'] = pd.to_datetime(df_all['rev_timestamp'], format="%Y%m%d%H%M%S")

df_all = df_all.set_index('reverting_timestamp_dt')

df_all['reverting_timestamp_dt'] = pd.to_datetime(df_all['reverting_timestamp'], format="%Y%m%d%H%M%S")

df_all['time_to_revert'] = df_all['reverting_timestamp_dt']-df_all['reverted_timestamp_dt']

df_all['time_to_revert_hrs'] = df_all['time_to_revert'].astype('timedelta64[s]')/(60*60)

df_all['time_to_revert_days'] = df_all['time_to_revert'].astype('timedelta64[s]')/(60*60*24)

df_all['reverting_year'] = df_all['reverting_timestamp_dt'].apply(get_year)

df_all['time_to_revert_days_log10'] = df_all['time_to_revert_days'].apply(np.log10)

df_all['time_to_revert_hrs_log10'] = df_all['time_to_revert_hrs'].apply(np.log10)

In [60]:
df_all[df_all['language']=='en'].reverting_year.value_counts().sort_index()

2004         2
2005       131
2006      3119
2007     17042
2008     33114
2009     36423
2010     30890
2011     63407
2012     48042
2013    201562
2014     20594
2015     24597
2016     21630
Name: reverting_year, dtype: int64

In [61]:
df_all.reverting_year.value_counts().sort_index()

2003         2
2004       131
2005      1553
2006      7122
2007     29852
2008     53571
2009     78903
2010     64994
2011    137364
2012     93883
2013    324587
2014     25804
2015    174120
2016     24367
2017       554
Name: reverting_year, dtype: int64

### Truncate to 2016-12-01 for consisntency acrosss datasets

In [62]:
df_all = df_all.ix["2001-01-01":"2016-12-01"].copy()

In [63]:
df_all.reverting_year.value_counts().sort_index()

2003         2
2004       131
2005      1553
2006      7122
2007     29852
2008     53571
2009     78903
2010     64994
2011    137364
2012     93883
2013    324587
2014     25804
2015    174120
2016     24160
Name: reverting_year, dtype: int64

### Check time to revert for negatives

In [64]:
len(df_all.query("time_to_revert_days < 0"))

160019

In [65]:
len(df_all.query("time_to_revert_days > 0"))

856027

In [66]:
df_all.query("time_to_revert_days < 0").groupby("language")['rev_id'].count()

language
de    101721
es       195
fr        68
ja     58035
Name: rev_id, dtype: int64

In [67]:
df_all.query("time_to_revert_days > 0").groupby("language")['rev_id'].count()

language
de     31870
en    500544
es     88347
fr     96404
ja     16497
pt     70948
zh     51417
Name: rev_id, dtype: int64

In [68]:
df_all.query("time_to_revert_days < 0").sample(1000)

,archived,language,page_namespace,rev_deleted,rev_id,rev_minor_edit,rev_page,rev_parent_id,rev_revert_offset,rev_sha1,rev_timestamp,rev_user,rev_user_text,reverted_to_rev_id,reverting_archived,reverting_comment,reverting_deleted,reverting_id,reverting_minor_edit,reverting_page,reverting_parent_id,reverting_sha1,reverting_timestamp,reverting_user,reverting_user_text,revisions_reverted,namespace_type,reverted_timestamp_dt,reverting_timestamp_dt,time_to_revert,time_to_revert_hrs,time_to_revert_days,reverting_year,time_to_revert_days_log10,time_to_revert_hrs_log10
reverting_timestamp_dt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-04-21 11:01:55,False,de,3,False,141346293,False,8738965,141169631.0,1,j8xonfr4vboaaxvuntxlhdcg3nn4svo,20150422092623,1768646,MediaWiki message delivery,140062634,False,Maintenance script verschob die Seite [[Benutz...,False,141169631,True,8738965,140062634.0,hdodejjfizauxnidqkdixo88lrzth45,20150421110155,2140963,Maintenance script,1,other talk,2015-04-22 09:26:23,2015-04-21 11:01:55,-1 days +01:35:32,-22.407778,-0.933657,2015,NaN,NaN
2012-12-04 15:45:38,False,de,0,False,116639725,True,4040283,112425366.0,13,iwarmocg58hlcsty11l149vz36p136w,20130402145058,901616,EmausBot,111754943,False,r2.7.1) (Bot: Ergänze: [[az:Dev Patel]],False,111282956,True,4040283,109882269.0,60th2tznci4577gshr5reghxqubjvm7,20121204154538,1164228,Kasirbot,13,article,2013-04-02 14:50:58,2012-12-04 15:45:38,-119 days +00:54:40,-2855.088889,-118.962037,2012,NaN,NaN
2015-04-20 03:44:09,False,ja,3,False,55269788,False,3167420,55201470.0,1,7uujryrtyj9avbd1cig2uaequw8qpni,20150421190138,791034,MediaWiki message delivery,54778785,False,Maintenance script がページ「[[利用者‐会話:Lavie]]」を「[[利...,False,55201470,True,3167420,54778785.0,4cec50zrxi72khl88hp40s1kq40lhex,20150420034409,962693,Maintenance script,1,other talk,2015-04-21 19:01:38,2015-04-20 03:44:09,-2 days +08:42:31,-39.291389,-1.637141,2015,NaN,NaN
2015-04-20 04:17:53,False,ja,3,False,55289977,False,3180958,55223194.0,1,jdxv8upizoja3ejiu5608js9e8abxxn,20150421192449,791034,MediaWiki message delivery,54801092,False,Maintenance script がページ「[[利用者‐会話:Tomg]]」を「[[利用...,False,55223194,True,3180958,54801092.0,tcpfjci3koamfz2mt76rdoy735kwsud,20150420041753,962693,Maintenance script,1,other talk,2015-04-21 19:24:49,2015-04-20 04:17:53,-2 days +08:53:04,-39.115556,-1.629815,2015,NaN,NaN
2009-02-27 11:08:30,False,ja,0,False,28154850,True,49606,27980070.0,1,f2sfp2z02rrycz40i3zxzmvwr8p4m5c,20090924170058,265089,ArthurBot,23550524,False,ロボットによる 除去: [[bh:Flag of India]],False,24598674,True,49606,24449069.0,kvahccp14tufqy74830acpby1hh4out,20090227110830,272877,Darkicebot,1,article,2009-09-24 17:00:58,2009-02-27 11:08:30,-210 days +18:07:32,-5021.874444,-209.244769,2009,NaN,NaN
2015-04-20 03:52:23,False,ja,3,False,55274855,False,3173322,55206858.0,1,aptp7psvcnt78iljttuo2rxqdt2mvte,20150421190838,791034,MediaWiki message delivery,54788343,False,Maintenance script がページ「[[利用者‐会話:Moron]]」を「[[利...,False,55206858,True,3173322,54788343.0,3a32nfbpwqhiiw9o95fsqa7julmweaj,20150420035223,962693,Maintenance script,1,other talk,2015-04-21 19:08:38,2015-04-20 03:52:23,-2 days +08:43:45,-39.270833,-1.636285,2015,NaN,NaN
2015-04-21 10:53:03,False,de,3,False,141339833,False,8732896,141162582.0,1,224mqqwngzhhiel6bnozefnvarxuej3,20150422092010,1768646,MediaWiki message delivery,140055487,False,Maintenance script verschob die Seite [[Benutz...,False,141162582,True,8732896,140055487.0,slb3clxa1cfbkhtm2nvioi5ag7retss,20150421105303,2140963,Maintenance script,1,other talk,2015-04-22 09:20:10,2015-04-21 10:53:03,-1 days +01:32:53,-22.451944,-0.935498,2015,NaN,NaN
2015-04-21 11:05:18,False,de,3,False,141348436,False,8735039,141171900.0,1,js9ucnqa78lj309oeh1rdr68dhwyxj9,20150422092912,1768646,MediaWiki message delivery,140058029,False,Maintenance script verschob die Seite [[Benutz...,False,141171900,True,8735039,140058029.0,jve5wtnjfw0avug8hu4m2g867q70osb,20150421110518,2140963,Maintenance script,1,other talk,2015-04-22 09:29:12,2015-04-21

In [69]:
df_all.query("time_to_revert_days < 0").to_csv("20170420_negative_ttr.tsv", sep="\t")

In [ ]:
!bzip2 -k 20170420_negative_ttr.tsv

## Other processing and metadata

### Botpair -- [bot1] rv [bot2]

In [29]:
def concat_botpair(row):
    return str(row['reverting_user_text']) + " rv " + str(row['rev_user_text'])

def sorted_botpair(row):
    """
    Returns a sorted list. list.sort() is locale dependent, but it doesn't matter
    because all we need is some consistent (if arbitrary) way of uniquely sorting.
    """
    
    return str(sorted([row['reverting_user_text'], row['rev_user_text']]))

In [30]:
df_all['botpair'] = df_all.apply(concat_botpair, axis=1)

/home/staeiou/conda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [31]:
df_all['botpair_sorted'] = df_all.apply(sorted_botpair, axis=1)

/home/staeiou/conda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### Namespace type

In [32]:
def namespace_type(item):
    if int(item) == 0:
        return 'article'
    elif int(item) == 14:
        return 'category'
    elif int(item) % 2 == 1:
        return 'other talk'
    else:
        return 'other page'

In [33]:
df_all['namespace_type'] = df_all['page_namespace'].apply(namespace_type)

/home/staeiou/conda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [34]:
df_all['namespace_type'].value_counts()

article       525163
other talk    201380
category      175691
other page    113812
Name: namespace_type, dtype: int64

### Reverts per page per botpair

In [35]:
gb_lpb = df_all.groupby(["language", "rev_page", "botpair"])
gb_lpb_s = df_all.groupby(["language", "rev_page", "botpair_sorted"])

In [36]:
df_lpb = pd.DataFrame(gb_lpb['rev_id'].count()).reset_index().rename(columns={"rev_id":"reverts_per_page_botpair"})
df_lpb[0:5]

,language,rev_page,botpair,reverts_per_page_botpair
0,de,3,Jotterbot rv Luckas-bot,1
1,de,5,TXiKiBoT rv VolkovBot,1
2,de,82,Alecs.bot rv SieBot,1
3,de,88,WikitanvirBot rv TobeBot,1
4,de,96,YurikBot rv RobotQuistnix,1


In [37]:
df_lpb_s = pd.DataFrame(gb_lpb_s['rev_id'].count()).reset_index().rename(columns={"rev_id":"reverts_per_page_botpair_sorted"})
df_lpb_s[0:5]

,language,rev_page,botpair_sorted,reverts_per_page_botpair_sorted
0,de,3,"['Jotterbot', 'Luckas-bot']",1
1,de,5,"['TXiKiBoT', 'VolkovBot']",1
2,de,82,"['Alecs.bot', 'SieBot']",1
3,de,88,"['TobeBot', 'WikitanvirBot']",1
4,de,96,"['RobotQuistnix', 'YurikBot']",1


In [38]:
df_all = pd.merge(df_all, df_lpb, how='left',
         left_on=["language", "rev_page", "botpair"],
         right_on=["language", "rev_page", "botpair"])

df_all = pd.merge(df_all, df_lpb_s, how='left',
         left_on=["language", "rev_page", "botpair_sorted"],
         right_on=["language", "rev_page", "botpair_sorted"])

### Final data format

In [42]:
len(df_all)

1016046

In [43]:
df_all[0:2].transpose()

,0,1
archived,False,False
language,de,de
page_namespace,0,0
rev_deleted,False,False
rev_id,75581173,45840823
rev_minor_edit,True,True
rev_page,3,5
rev_parent_id,7.555e+07,4.57112e+07
rev_revert_offset,6,3
rev_sha1,rnc3xl21j407j5hdvzwyoy981nhf9xy,53c0cu3qanye8bnugdvu98a7ipxexys


## Output to file

In [44]:
df_all.to_pickle("../../datasets/pandas_df_all_2017.pickle")

In [45]:
!bzip2 -k ../../datasets/pandas_df_all.pickle